
# [Screening REITs (Real Estate Investment Trusts) Using Financial Ratios with OpenBB]

This notebook serves as a guide for screening Real Estate Investment Trusts (REITs) using financial metrics, leveraging data provided by OpenBB. The purpose of screening is to identify potential investment opportunities by filtering REITs that meet certain financial criteria. The notebook outlines a step-by-step approach to retrieve REIT data, apply screening criteria based on financial ratios, and filter the REITs that satisfy these criteria.

##Key Steps

**Data Acquisition:**

Retrieve financial data for a list of REITs using OpenBB, focusing on metrics such as Price-to-Earnings (P/E) ratio, dividend yield, and Funds From Operations (FFO).

**Screening Criteria Definition:**

Set up conditions for filtering REITs:
P/E ratio below 60 to ensure reasonable valuation.
Dividend yield above 2%, indicating income potential.
Positive FFO, signifying adequate cash flow.

**Filtering and Output:**

Apply the criteria to filter out REITs that meet all the conditions and display the results in a structured format.
Optional Analysis and Visualization:
Optionally, include visualizations like scatter plots or bar charts to analyze the screened REITs further.

# Author
[The Fool 76](https://github.com/thefool76)

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1vVf8SDXDdpLW-MLjNmmwrPEntVl19bJ9?usp=sharing)




## Installation and Setup

To use the OpenBB, ensure that the following packages are installed. If you haven't installed them already, use the following command:

```bash
!pip install openbb pandas numpy
```

Once installed, you can proceed with the imports and configurations.


In [11]:
!pip install openbb
!pip install pandas
!pip install numpy

In [13]:
import pandas as pd
import numpy as np
from openbb import obb

In [14]:
# Define a list of REITs (tickers from major exchanges)
reit_symbols = ["O", "SPG", "DLR", "PLD", "AMT", "ARE"]

# OpenBB and FMP setup
Step 1: Login to OpenBB using a Personal Access Token (PAT)
To authenticate and access the OpenBB hub, you need to log in using a Personal Access Token (PAT). Replace the placeholder PAT string in the code with your actual token.
Important: Ensure you handle this securely, either by storing it in environment variables or using a secure method to avoid exposing it directly in your code.

	•	pat: This is your unique personal access token (PAT) for OpenBB.
	•	remember_me: Set this to True if you want the session to persist beyond the current runtime.

Step 2: Set up Financial Modeling Prep (FMP) API key
To access the Financial Modeling Prep (FMP) API through OpenBB, you need to provide your FMP API key. Replace the placeholder string with your actual API key. Like with the PAT, it’s best to store it securely.

	•	fmp_api_key: This is your Financial Modeling Prep API key, which allows access to financial data via OpenBB.

Step 3: Save Account Changes
Once logged in and API keys are set, save the changes to persist your session and credentials in the OpenBB environment. This ensures that the login and API key are stored and used for future commands in the session.


In [15]:
# Step 1: Login to OpenBB using PAT
# Note: For security, replace with your personal access token (PAT) securely
obb.account.login(pat="<Your PAT>", remember_me=True)  # Replace with your personal access token (PAT)

# Step 2: Set up Financial Modeling Prep (FMP) API key
obb.user.credentials.fmp_api_key = "<FMP API KEY>"  # Replace with your actual FMP API key

# Save account changes
obb.account.save()

The get_reit_financials function retrieves and processes financial data for a list of Real Estate Investment Trusts (REITs). It uses the OpenBB SDK to fetch key financial metrics like dividend yield, price-to-earnings (P/E) ratio, and funds from operations (FFO) per share. Here’s how the function works:

**Initialize a Dictionary:**
The function starts by creating an empty dictionary, reit_data, to store the extracted financial metrics for each REIT symbol.

**Loop Over REIT Symbols**
For each symbol in the provided list:
* It attempts to fetch financial data using the OpenBB lib's equity.fundamental.metrics method, specifying Financial Modeling Prep (FMP) as the data provider.
* If no data is found for a particular REIT symbol, a message is printed, and the REIT is skipped.

**Extract Relevant Financial Metrics**
For each symbol that returns data, the function attempts to extract:
*	Dividend Yield: If available, the most recent non-null value is retrieved.
*	Price-to-Earnings Ratio (P/E Ratio): Similarly, the function looks for the most recent value if the data is available.
*	Funds from Operations (FFO) Per Share: The function retrieves the latest available value for free cash flow per share, which serves as a proxy for FFO.
If any of these metrics are missing from the data, NaN values are assigned to ensure consistent output.

**Error Handling**
In case of an error (e.g., network issues, unavailable data), the function catches the exception, prints an error message, and assigns NaN to all the metrics for that symbol.

**Return Data as DataFrame**
After processing all the symbols, the function returns a pandas DataFrame where:
* Each row corresponds to a REIT symbol.
* Columns represent the extracted metrics: dividend_yield,  price_earnings_ratio, and funds_from_operations.


Step 3: Fetch REIT Financial Data

The list of REIT symbols, reit_symbols, is passed to the function to fetch the relevant financial data. The resulting DataFrame is stored in reit_df and displayed.


In [16]:

def get_reit_financials(symbols):
    reit_data = {}
    for symbol in symbols:
        try:
            # Fetch financial metrics (dividend yield, P/E ratio, FFO, etc.)
            financials = obb.equity.fundamental.metrics(symbol, provider="fmp", period="annual", limit=5).to_df()

            if financials.empty:
                print(f"No financial data found for {symbol}. Skipping.")
                continue

            # Extract relevant columns if available
            dividend_yield = financials.get('dividend_yield').dropna().values[-1] if 'dividend_yield' in financials else np.nan
            price_earnings_ratio = financials.get('pe_ratio').dropna().values[-1] if 'pe_ratio' in financials else np.nan
            funds_from_operations = financials.get('free_cash_flow_per_share').dropna().values[-1] if 'free_cash_flow_per_share' in financials else np.nan

            # Store the extracted metrics in the dictionary
            reit_data[symbol] = {
                'dividend_yield': dividend_yield,
                'price_earnings_ratio': price_earnings_ratio,
                'funds_from_operations': funds_from_operations
            }

        except Exception as e:
            print(f"An error occurred while fetching data for {symbol}: {e}")
            reit_data[symbol] = {'dividend_yield': np.nan, 'price_earnings_ratio': np.nan, 'funds_from_operations': np.nan}

    # Return the collected data as a DataFrame
    return pd.DataFrame(reit_data).transpose()

# Step 3: Fetch the financial data for the list of REITs
reit_df = get_reit_financials(reit_symbols)

# Display the collected data
print(reit_df)


     dividend_yield  price_earnings_ratio  funds_from_operations
O          0.037840             51.592457               3.309938
SPG        0.055826             21.827464               9.520438
DLR        0.039959             43.026237               0.052019
PLD        0.023256             36.786141               2.991301
AMT        0.015769             53.847734               6.242780
ARE        0.024885            101.084572               6.094765


**Filtering REITs Based on Financial Criteria**

After retrieving the financial data for the REITs, a set of criteria is applied to filter out those that meet specific thresholds for financial health and performance.

**Price-to-Earnings Ratio (P/E Ratio)**
The REIT must have a price-to-earnings ratio less than 60. This increased threshold aims to allow slightly higher P/E ratios, accommodating REITs that might be growing or have a premium valuation but still provide value.

**Dividend Yield**
The dividend yield must be greater than 2% (0.02). This lowered threshold ensures that the REITs offer a reasonable yield, focusing on income-generating properties.

**Funds from Operations (FFO)**
The FFO per share must be greater than 0. This ensures the REIT is generating positive cash flow from its core operations, which is critical for assessing financial stability.

**Filtering Process**

The criteria are applied using the pandas DataFrame reit_df, which contains the financial metrics for each REIT.
The filtered DataFrame screened_reits contains only the REITs that satisfy all three of the defined criteria. Each REIT that passes this screen should:
*  Have a manageable P/E ratio (under 60),
*  Offer a reasonable dividend yield (above 2%),
*  Show positive funds from operations (greater than 0).

Once filtered, the REITs that meet the criteria are stored in screened_reits and displayed. This allows for further analysis or decision-making based on the screened results.


In [17]:
criteria = (
    (reit_df["price_earnings_ratio"] < 60) &  # Increased threshold
    (reit_df["dividend_yield"] > 0.02) &      # Lowered threshold
    (reit_df["funds_from_operations"] > 0.0)
)

# Filter REITs that meet the new criteria
screened_reits = reit_df[criteria]

# Display the filtered REITs
screened_reits




,dividend_yield,price_earnings_ratio,funds_from_operations
O,0.037840,51.592457,3.309938
SPG,0.055826,21.827464,9.520438
DLR,0.039959,43.026237,0.052019
PLD,0.023256,36.786141,2.991301
